# Spark SQl 
Processing-CPU LOGS 

In [1]:
import os
os.environ["SPARK_LOCAL_IP"] = "10.0.2.15"

1. Find maximum idle hours

In [2]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark import SparkContext as sc
import pyspark.sql.functions 

In [3]:
# create spark session

spark = SparkSession.builder.master('local[*]').appName("MaxIdleHours").config("spark.jars","/opt/spark/jars/mysql-connector-j-8.0.33.jar").getOrCreate()


23/07/19 16:22:13 WARN Utils: Your hostname, UbuntuOS resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/19 16:22:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/19 16:22:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/19 16:22:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:

# Reading CSV file from hadoop
csv_file_path1 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-16.csv"

csv_file_path2 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-17.csv"
csv_file_path3 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-18.csv"
csv_file_path4 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-19.csv"
csv_file_path5 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-20.csv"
csv_file_path6 = "hdfs://localhost:9000/pyspark_sql/csv_cpu_logs/CpuLogData2019-09-21.csv"



# sc.setLogLevel(self,logLevel='Debug')

df1 = spark.read.csv(csv_file_path1,header=True)
df2 = spark.read.csv(csv_file_path2,header=True)
df3 = spark.read.csv(csv_file_path3,header=True)
df4 = spark.read.csv(csv_file_path4,header=True)
df5 = spark.read.csv(csv_file_path5,header=True)
df6 = spark.read.csv(csv_file_path6,header=True)


In [5]:
# union all dataframes

combined_csv_df = df1.union(df2).union(df3).union(df4).union(df5).union(df6)

combined_csv_df.head(10)

combined_csv_df.count()

23/07/19 15:59:11 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


4122

In [6]:

csv_columns = combined_csv_df.columns

print(csv_columns)



['DateTime', 'Cpu Count', 'Cpu Working Time', 'Cpu idle Time', 'cpu_percent', 'Usage Cpu Count ', 'number of software interrupts since boot', 'number of system calls since boot', 'number of interrupts since boot', 'cpu avg load over 1 min', 'cpu avg load over 5 min', 'cpu avg load over 15 min', 'system_total_memory', 'system_used_memory', 'system_free_memory', 'system_active_memory', 'system_inactive_memory', 'system_buffers_memory', 'system_cached_memory', 'system_shared_memory', 'system_avalible_memory', 'disk_total_memory', 'disk_used_memory', 'disk_free_memory', 'disk_read_count', 'disk_write_count', 'disk_read_bytes', 'disk_write_bytes', 'time spent reading from disk', 'time spent writing to disk', 'time spent doing actual I/Os', 'number of bytes sent', 'number of bytes received', 'number of packets sent', 'number of packets recived', 'total number of errors while receiving', 'total number of errors while sending', 'total number of incoming packets which were dropped', 'total numb

In [7]:
# create tem view
combined_csv_df.createOrReplaceTempView("combined_csv_data")

In [8]:
sql_query = spark.sql("SELECT user_name, MAX(idle_time) AS max_idle_time FROM (SELECT user_name, Datetime,UNIX_TIMESTAMP(Datetime) - UNIX_TIMESTAMP(LAG(Datetime) OVER (PARTITION BY user_name ORDER BY Datetime)) AS idle_time FROM combined_csv_data WHERE keyboard = '0.0' AND mouse = '0.0') subquery GROUP BY user_name ORDER BY max_idle_time desc")

sql_query.show()

+--------------------+-------------+
|           user_name|max_idle_time|
+--------------------+-------------+
|deepshukla292@gma...|       141300|
|markfernandes66@g...|        85500|
|sharlawar77@gmail...|        84300|
|rahilstar11@gmail...|        83400|
|bhagyashrichalke2...|        83400|
|salinabodale73@gm...|        82500|
|damodharn21@gmail...|        80100|
|  iamnzm@outlook.com|        75600|
+--------------------+-------------+



2. find lowest number of average hours

In [9]:
sql_query1 = spark.sql("""
  SELECT user_name, AVG(hour(DateTime)) AS min_avg_hours
  FROM combined_csv_data where mouse !=0 and keyboard !=0
  GROUP BY user_name
  ORDER BY min_avg_hours
""")

sql_query1.show()

+--------------------+------------------+
|           user_name|     min_avg_hours|
+--------------------+------------------+
|  iamnzm@outlook.com|14.090140845070422|
|bhagyashrichalke2...|          14.09375|
|deepshukla292@gma...|14.129449838187702|
|salinabodale73@gm...|14.172727272727272|
|markfernandes66@g...|14.256410256410257|
|sharlawar77@gmail...|14.417910447761194|
|rahilstar11@gmail...|14.586440677966102|
|damodharn21@gmail...| 14.88888888888889|
+--------------------+------------------+



3. Find max number of average hours

In [10]:
sql_query2 = spark.sql("""
  SELECT user_name, AVG(hour(DateTime)) AS max_avg_hours
  FROM combined_csv_data where keyboard !=0 and mouse !=0
  GROUP BY user_name
  ORDER BY max_avg_hours DESC
""")

sql_query2.show()

+--------------------+------------------+
|           user_name|     max_avg_hours|
+--------------------+------------------+
|damodharn21@gmail...| 14.88888888888889|
|rahilstar11@gmail...|14.586440677966102|
|sharlawar77@gmail...|14.417910447761194|
|markfernandes66@g...|14.256410256410257|
|salinabodale73@gm...|14.172727272727272|
|deepshukla292@gma...|14.129449838187702|
|bhagyashrichalke2...|          14.09375|
|  iamnzm@outlook.com|14.090140845070422|
+--------------------+------------------+



In [11]:
combined_csv_df.head(10)

[Row(DateTime='2019-09-16 12:55:03', Cpu Count='4', Cpu Working Time='1903.63', Cpu idle Time='20452.89', cpu_percent='6.1', Usage Cpu Count ='4', number of software interrupts since boot='5985878', number of system calls since boot='0', number of interrupts since boot='9940284', cpu avg load over 1 min='1.34', cpu avg load over 5 min='1.39', cpu avg load over 15 min='0.99', system_total_memory='8284991488', system_used_memory='6763266048', system_free_memory='121323520', system_active_memory='6131396608', system_inactive_memory='1458581504', system_buffers_memory='316784640', system_cached_memory='1083617280', system_shared_memory='521969664', system_avalible_memory='704999424', disk_total_memory='313445539840', disk_used_memory='32033652736', disk_free_memory='265418379264', disk_read_count='383708', disk_write_count='66449', disk_read_bytes='4290477056', disk_write_bytes='2469106688', time spent reading from disk='14529344', time spent writing to disk='16266860', time spent doing ac

4. Find users with highest number of late comings

In [15]:
sql_query3 = spark.sql("""
  SELECT user_name, `rank`
  FROM (
    SELECT user_name, RANK() OVER (ORDER BY DateTime DESC) AS `rank`
    FROM combined_csv_data
  ) ranked_data
  WHERE  `rank` = 1
""")

sql_query3.show()


23/07/19 16:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/19 16:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/19 16:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/19 16:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/07/19 16:00:49 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+--------------------+----+
|           user_name|rank|
+--------------------+----+
|sharlawar77@gmail...|   1|
+--------------------+----+



In [16]:
# Stroing data into sql database
import os
# Define the JDBC connection properties
try:
  database_url = "jdbc:mysql://localhost:3306/pysparksql"
  database_table = "idle_cpu_users"
  database_properties = {
      "user": os.environ['DB_USERNAME'],
      "password": os.environ['DB_PASSWORD'],
      "driver": "com.mysql.cj.jdbc.Driver"
  }

  # Write the DataFrame to the SQL database
  sql_query.write \
    .format("jdbc") \
    .option("url", database_url) \
    .option("dbtable", database_table) \
    .option("user", database_properties["user"]) \
    .option("password", database_properties["password"]) \
    .option("driver", database_properties["driver"]) \
    .mode("overwrite") \
    .save()

except Exception as e:
  print(str(e))  
  


  
  


In [4]:
import os
# Read from SQL Table
df = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/pysparksql;databaseName='idle_cpu_users") \
  .option("dbtable", "select user_name from idle_cpu_users") \
  .option("user", os.environ['DB_USERNAME']) \
  .option("password", os.environ['DB_PASSWORD']) \
  .load()

df.show()


NameError: name 'os' is not defined